In [6]:
#Imports
import time
import requests
from urllib.parse import urlparse, urljoin
from bs4 import BeautifulSoup
# !pip install selenium
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
import os
import urllib.request
from urllib.request import urlopen
import numpy as np
import pandas as pd
from pandas import DataFrame
# import pyautogui
from requests.exceptions import ConnectionError

In [7]:
# from google.colab import drive
# drive.mount('/content/drive')

In [8]:
internal_urls = set()
external_urls = set()

In [9]:
def is_url_valid(url):
   #Checks if the url is valid or not
    parsedURL = urlparse(url)
    return bool(parsedURL.netloc) and bool(parsedURL.scheme)

#The above function works as follows. We check if the scheme is present and there is a value in the network location part
# url = "https://umd.edu/virusinfo"
# urlparse(url)
# ParseResult(scheme='https', netloc='umd.edu', path='/virusinfo', 
# params='', query='', fragment='')

In [10]:
#The function gives all urls
count = 0
def get_all_urls(url):
    urls = set()
    # domain name of the URL without the protocol (umd.edu in this case)
    domain_name = urlparse(url).netloc
    soup = BeautifulSoup(requests.get(url).content, "html.parser")
    for a_tag in soup.findAll("a"):
        href = a_tag.attrs.get("href")
        if href == "" or href is None:
            #href is empty and we don't need that a element
            continue
        #if the link is not absolute, make it by joining relative to the base
        href = urljoin(url, href)
        parsed_href = urlparse(href)
        #constructing an absolute URL from parsed data
        href = parsed_href.scheme + "://" + parsed_href.netloc + parsed_href.path
        if not is_url_valid(href):
            #in valid url
            continue
        if href in internal_urls:
            #it is already in the set, so we don't need to add
            continue
        if domain_name not in href:
            #it is an external link. i.e
            # Check if it is already there 
            if href not in external_urls:
#                 print(f"[EXT] External link: {href}")
                external_urls.add(href)
            continue
#         print(".",end=" ")
        global count
        count = count+1
        print(f"{count}[INT] Internal link: {href}")

        urls.add(href)
        internal_urls.add(href)
    return urls

In [11]:
total_urls_visited = 0
def crawl(url, max_urls=20000):
    #Max URL is just to decrease the time if there are a lot of pages.
    #The following code was openly available of github and I found this
    #idea useful to inhibit crawling time
    global total_urls_visited
    total_urls_visited += 1
    try:
        links = get_all_urls(url)
        for link in links:
            if total_urls_visited > max_urls:
                break
            crawl(link, max_urls=max_urls)
    except Exception as e:
        print(e)

In [12]:
#base_url = "https://umd.edu/virusinfo"
#Input
# base_url = input("Enter the URL : ")
base_url = "https://www.archanaskitchen.com/"
internal_urls.add(base_url)

In [13]:
parsedurl = urlparse(base_url)
print(parsedurl)
base_url_main = parsedurl.scheme+"://"+parsedurl.netloc
print(base_url_main)
# base_url_text = base_url.split("//",1)[1]
base_url_text = parsedurl.netloc+parsedurl.path
print(base_url_text)
# base_url_text_domain = base_url_text.split("/",1)[0]
base_url_text_domain = parsedurl.netloc
print(base_url_text_domain)

ParseResult(scheme='https', netloc='www.archanaskitchen.com', path='/', params='', query='', fragment='')
https://www.archanaskitchen.com
www.archanaskitchen.com/
www.archanaskitchen.com


In [ ]:
crawl(base_url)
print("[+] Total External links:", len(external_urls))
print("[+] Total Internal links:", len(internal_urls))
print("[+] Total:", len(external_urls) + len(internal_urls))

1[INT] Internal link: https://www.archanaskitchen.com/about-archana
2[INT] Internal link: https://www.archanaskitchen.com/contact-us
3[INT] Internal link: https://www.archanaskitchen.com/advertise
4[INT] Internal link: https://www.archanaskitchen.com/archanas-kitchen-partners
5[INT] Internal link: https://www.archanaskitchen.com/brands
6[INT] Internal link: https://www.archanaskitchen.com/media-coverage
7[INT] Internal link: https://www.archanaskitchen.com/recipes
8[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes
9[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes/indian-snack-recipes
10[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes/homemade-energy-bar-recipes
11[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes/sandwich-recipes
12[INT] Internal link: https://www.archanaskitchen.com/recipes/snack-recipes/burger-recipes
13[INT] Internal link: https://www.archanaskitchen.com/recipes/snac

182[INT] Internal link: https://www.archanaskitchen.com/tomato-onion-cucumber-raita
183[INT] Internal link: https://www.archanaskitchen.com/nellikai-chitranna
184[INT] Internal link: https://www.archanaskitchen.com/mavinakayi-chitranna-recipe-spiced-raw-mango-tempered-rice
185[INT] Internal link: https://www.archanaskitchen.com/kayi-saasive-chitranna-recipe
186[INT] Internal link: https://www.archanaskitchen.com/cuisine/south-indian
187[INT] Internal link: https://www.archanaskitchen.com/tag/south-indian-recipes
188[INT] Internal link: https://www.archanaskitchen.com/tag/festival-recipes
189[INT] Internal link: https://www.archanaskitchen.com/tag/gluten-free-recipes
190[INT] Internal link: https://www.archanaskitchen.com/tag/ugadi-recipes
191[INT] Internal link: https://www.archanaskitchen.com/tag/bachelor-recipes
192[INT] Internal link: https://www.archanaskitchen.com/tag/dussehra-festival-recipes
193[INT] Internal link: https://www.archanaskitchen.com/tag/recipe-for-beginners
194[INT

277[INT] Internal link: https://www.archanaskitchen.com/brands/preethi-kitchen-appliances
278[INT] Internal link: https://www.archanaskitchen.com/phulka-roti-chapati-puffed-indian-bread
279[INT] Internal link: https://www.archanaskitchen.com/punjabi-dal-tadka
280[INT] Internal link: https://www.archanaskitchen.com/kachumber-salad-recipe-with-cucumber-onion-tomatoes
281[INT] Internal link: https://www.archanaskitchen.com/potato-vindaloo-recipe-spicy-goan-potato-curry
282[INT] Internal link: https://www.archanaskitchen.com/sweet-potato-tangy-salad-chaat
283[INT] Internal link: https://www.archanaskitchen.com/tamarind-chilli-potato-recipe
284[INT] Internal link: https://www.archanaskitchen.com/potato-achari-recipe-in-hindi
285[INT] Internal link: https://www.archanaskitchen.com/tag/side-dish-recipes
286[INT] Internal link: https://www.archanaskitchen.com/tag/lunch-recipe-ideas
287[INT] Internal link: https://www.archanaskitchen.com/tag/potato-recipes
288[INT] Internal link: https://www.ar

430[INT] Internal link: https://www.archanaskitchen.com/spinach-and-cottage-cheese-cannelloni-in-roasted-pepper-sauce-recipe
431[INT] Internal link: https://www.archanaskitchen.com/tag/capsicum-bell-pepper-recipe
432[INT] Internal link: https://www.archanaskitchen.com/prawns-and-spinach-spaghetti-recipe
433[INT] Internal link: https://www.archanaskitchen.com/tag/prawns-recipe
434[INT] Internal link: https://www.archanaskitchen.com/roasted-vegetable-flan-recipe-in-herbed-sauce-european-style-baked-vegetables
435[INT] Internal link: https://www.archanaskitchen.com/black-eyed-bean-and-lemon-bruschetta-recipe
436[INT] Internal link: https://www.archanaskitchen.com/sweet-potato-gnocchi-with-garlic-sweet-herb-butter-recipe
437[INT] Internal link: https://www.archanaskitchen.com/tag/sweet-potato-recipes
438[INT] Internal link: https://www.archanaskitchen.com/focaccia-with-cherry-tomatoes-caramelized-onion-recipe
439[INT] Internal link: https://www.archanaskitchen.com/eggplant-crostini-brusche

530[INT] Internal link: https://www.archanaskitchen.com/brands/future-consumer
531[INT] Internal link: https://www.archanaskitchen.com/sakkarai-pongal-sweetened-rice-with-jaggery
532[INT] Internal link: https://www.archanaskitchen.com/video-recipe-vada-sambar-non-fried-lentil-fritters-with-lentil-curry
533[INT] Internal link: https://www.archanaskitchen.com/kathirikai-gothsu-spicy-and-tangy-eggplant-curry
534[INT] Internal link: https://www.archanaskitchen.com/thakkali-rasam-recipe-south-indian-tomato-rasam
535[INT] Internal link: https://www.archanaskitchen.com/video-recipe-coconut-chutney-a-delicious-south-indian-accompaniment
536[INT] Internal link: https://www.archanaskitchen.com/sambar-powder-podi
537[INT] Internal link: https://www.archanaskitchen.com/south-indian-rice-and-lentil-pudding-recipe-in-hindi
538[INT] Internal link: https://www.archanaskitchen.com/tag/high-protein-vegetarian-diet-recipes
539[INT] Internal link: https://www.archanaskitchen.com/tag/south-indian-breakfast

...

5745[INT] Internal link: http://www.archanaskitchen.com/idli-dosa-milagai-podi-chilli-chutney-powder
5746[INT] Internal link: http://www.archanaskitchen.com/carrot-onion-uttapam-recipe
5747[INT] Internal link: http://www.archanaskitchen.com/tomato-uttapam
5748[INT] Internal link: http://www.archanaskitchen.com/karamani-sundal-uttapam-recipe
5749[INT] Internal link: http://www.archanaskitchen.com/apple-and-raisin-chutney-recipe
5750[INT] Internal link: http://www.archanaskitchen.com/no-cream-healthy-broccoli-pasta-recipe
5751[INT] Internal link: http://www.archanaskitchen.com/vegetable-rava-upma-savory-semolina-breakfast-pudding
5752[INT] Internal link: http://www.archanaskitchen.com/vegetable-uttapam
5753[INT] Internal link: http://www.archanaskitchen.com/how-to-make-ghee-masala-dosa
5754[INT] Internal link: http://www.archanaskitchen.com/video-recipe-how-to-make-homemade-idlis-steamed-rice-and-lentil-cake
5755[INT] Internal link: http://www.archanaskitchen.com/the-paruppu-podi-recipe-

5826[INT] Internal link: http://www.archanaskitchen.com/cold-sesame-noodles-recipe
5827[INT] Internal link: http://www.archanaskitchen.com/sichuan-dan-dan-noodles-recipe-sesame-noodles-with-crispy-tofu-recipe
5828[INT] Internal link: http://www.archanaskitchen.com/spicy-schezuan-vegetarian-noodles-recipe
5829[INT] Internal link: http://www.archanaskitchen.com/chilli-potato-wedges
5830[INT] Internal link: http://www.archanaskitchen.com/indo-chinese-mixed-vegetables-in-szechuan-sauce
5831[INT] Internal link: http://www.archanaskitchen.com/tag/chinese-recipes
5832[INT] Internal link: http://www.archanaskitchen.com/tag/indo-chinese-recipes
5833[INT] Internal link: http://www.archanaskitchen.com/lip-smacking-vegetarian-indo-chinese-recipes-for-your-weekend-meals
5834[INT] Internal link: http://www.archanaskitchen.com/sweet-and-spicy-schezwan-style-vegetables-in-ginger-garlic-sauce
5835[INT] Internal link: http://www.archanaskitchen.com/potato-achari-recipe-achari-aloo
5836[INT] Internal lin

5916[INT] Internal link: https://www.archanaskitchen.com/tag/stir-fried-recipes/page-3
5917[INT] Internal link: http://www.archanaskitchen.com/chicken-sausages-baked-beans-and-stir-fried-broccoli-recipe
5918[INT] Internal link: http://www.archanaskitchen.com/potato-fenugreek-leaves-stir-fry-recipe-in-hindi
5919[INT] Internal link: http://www.archanaskitchen.com/mushroom-masala-with-corn-recipe
5920[INT] Internal link: http://www.archanaskitchen.com/asian-style-stir-fried-recipes-you-can-serve-as-side-dish-with-your-meals
5921[INT] Internal link: http://www.archanaskitchen.com/suprabha-s-matt
5922[INT] Internal link: http://www.archanaskitchen.com/himachali-bhey-recipe-stir-fried-lotus-stem-recipe
5923[INT] Internal link: http://www.archanaskitchen.com/bangaladumpa-vepudu-recipe-potato-fry-recipe
5924[INT] Internal link: http://www.archanaskitchen.com/aratikaaya-masala-pulusu-recipe-raw-banana-stir-fry-recipe
5925[INT] Internal link: http://www.archanaskitchen.com/andhra-style-chamadump

5999[INT] Internal link: https://www.archanaskitchen.com/cuisine/himachal
6000[INT] Internal link: https://www.archanaskitchen.com/tag/himachal-recipes
6001[INT] Internal link: https://www.archanaskitchen.com/pahadi-chicken-curry-recipe
6002[INT] Internal link: https://www.archanaskitchen.com/chole-capsicum-masala-recipe
6003[INT] Internal link: https://www.archanaskitchen.com/himachali-dham-a-festive-recipes-from-the-mountains-of-himachal-pradesh
6004[INT] Internal link: https://www.archanaskitchen.com/himachali-dry-raw-mango-chutney-recipe-in-hindi
6005[INT] Internal link: https://www.archanaskitchen.com/babru-recipe-himachali-black-gram-stuffed-bread
6006[INT] Internal link: https://www.archanaskitchen.com/himachali-pahadi-mutton-recipe
6007[INT] Internal link: https://www.archanaskitchen.com/himachal-style-khatta-jimikand-recipe-spicy-tangy-yam
6008[INT] Internal link: https://www.archanaskitchen.com/himachali-chana-madra-recipe-in-hindi
6009[INT] Internal link: https://www.archana

6076[INT] Internal link: https://www.archanaskitchen.com/aloo-simla-mirch-ki-sabzi-recipe-in-hindi
6077[INT] Internal link: https://www.archanaskitchen.com/one-pot-masala-vegetable-rice-recipe
6078[INT] Internal link: https://www.archanaskitchen.com/choto-aloor-dum-bengali-style-baby-potato-stir-fry
6079[INT] Internal link: https://www.archanaskitchen.com/aloo-baingan-ki-sabzi-recipe-in-hindi
6080[INT] Internal link: https://www.archanaskitchen.com/aloo-posto-recipe-potatoes-in-poppy-seed-paste
6081[INT] Internal link: https://www.archanaskitchen.com/choto-aloor-dum-recipe-bengali-style-baby-potato-stir-fry-in-hindi
6082[INT] Internal link: https://www.archanaskitchen.com/delicious-wholesome-lunch-box-masoor-dal-achari-aloo-and-jeera-rice
6083[INT] Internal link: https://www.archanaskitchen.com/baby-potato-recipes-that-you-can-make-for-your-everyday-meals
6084[INT] Internal link: https://www.archanaskitchen.com/tadkewali-masoor-dal
6085[INT] Internal link: https://www.archanaskitchen.c

In [ ]:
len(internal_urls)

In [29]:
setInternalUrls = set(internal_urls)
len(setInternalUrls)

22235

In [30]:
df = DataFrame(internal_urls,columns=['URL'])

In [31]:
df.shape

(22235, 1)

In [32]:
dfrecipe = df[df['URL'].str.contains('recipe')]

In [33]:
dfrecipe

,URL
0,https://www.archanaskitchen.com/masala-karela-...
1,http://www.archanaskitchen.com/spicy-tomato-ri...
3,http://www.archanaskitchen.com/ragi-vermicelli...
4,http://www.archanaskitchen.com/tag/kids-lunch-...
5,http://www.archanaskitchen.com/gongura-chicken...
...,...
22226,https://www.archanaskitchen.com/kerala-cheriya...
22227,http://www.archanaskitchen.com/pav-bhaji-flavo...
22230,https://www.archanaskitchen.com/tag/chilli-rec...
22231,https://www.archanaskitchen.com/keerai-masiyal...


In [34]:
dfrecipe.to_csv('urls_recipe10k.csv')